# Demand prediction baseline solution

Victor Kantor, xead.wl@gmail.com

In [190]:
import pandas as pd

train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")

In [198]:
from sklearn.model_selection import train_test_split

frac = 1 # fraction of learning examples used for model fitting

# sample the train set if your don't want to deel with all examples
train = train.sample(frac=frac, random_state=42)

X = train.drop(['Num','y'], axis=1)
y = train['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [178]:
important_features = ['week', 'item_id', 'shift', 'year', 'f1', 'f2', 'f3', 'f4', 'f5']

In [179]:
X_test = test[important_features]

In [180]:
xgbr = xgb.XGBRegressor(max_depth=15, n_estimators=1000, gamma=1)
xgbr.fit(X[important_features], y)
# preds = xgbr.predict()

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=1,
       learning_rate=0.1, max_delta_step=0, max_depth=15,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [181]:
new_predicitons = xgbr.predict(X_test)

In [206]:
def loss_func(F, A):
    if len(F) != len(A):
        raise BaseException
    res = 0
    for i in range(len(F)):
        res += float(abs(F[i] - A[i]) / (abs(A[i]) + abs(F[i])))
    res *= float(200) / len(F)
    return res

In [202]:
from sklearn.metrics import make_scorer
scorer = make_scorer(loss_func)

In [203]:
clf_config = {
    #'GradientBoostingRegressor' : (GradientBoostingRegressor(), {'n_estimators' : range(300,301), 'max_depth' : range(3,8)})
    'XGBRegressor' : (xgb.XGBRegressor(), {'max_depth' : range(3,10), 'n_estimators' : range(500, 2001, 100) })
}

In [204]:
from sklearn.model_selection import GridSearchCV

cv_values = {}
for name, (clf, args) in clf_config.items():
    print('Tuning parameters for %s' % name)
    gs_clf = GridSearchCV(estimator=clf, param_grid=args, scoring=scorer)
    gs_clf.fit(np.array(X[important_features]), np.array(y))
    
    print("\tBest params: {}".format(gs_clf.cv_results_))

Tuning parameters for XGBRegressor
	Best params: {'std_train_score': array([ 0.70312351,  0.10649143,  0.40530563,  0.16676921,  0.11318075,
        0.02699544,  0.11595556,  0.10025706,  0.03631183,  0.0875236 ,
        0.06463071,  0.04724396,  0.06057082,  0.07470113,  0.09432047,
        0.10801663,  1.53647239,  1.46533687,  1.68525279,  1.53527941,
        1.3264436 ,  1.2238285 ,  1.08102517,  1.02309306,  0.95040104,
        0.87125928,  0.84618422,  0.75666315,  0.77187812,  0.74312806,
        0.72382601,  0.75887967,  0.31994845,  0.39454166,  0.4513093 ,
        0.4940551 ,  0.5605404 ,  0.50857716,  0.44139417,  0.43780727,
        0.55789199,  0.56406889,  0.55871155,  0.54095002,  0.54468042,
        0.53586686,  0.5281029 ,  0.53144605,  0.34934559,  0.6234735 ,
        0.58199943,  0.60587194,  0.72084388,  0.72626002,  0.73038035,
        0.75705696,  0.76145796,  0.75530098,  0.75549887,  0.75159331,
        0.75423185,  0.74678938,  0.74051052,  0.73914061,  0.63959

In [205]:
#model = #GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=43)
# model = xgb.XGBRegressor(n_estimators=400, max_depth=10)
# model.fit(X_train, y_train)


In [104]:
preds = model.predict(np.array(test.drop(['Num'], axis=1)))

In [105]:
import numpy as np
preds = np.array(preds)
y_test = np.array(y_test)

In [106]:
loss_func(preds, y_test)

BaseException: 

In [183]:
sample_submission['y'] = new_predicitons

In [184]:
sample_submission.head(5)

,Num,y
0,348622,1467.554199
1,348623,16877.605469
2,348624,212119.640625
3,348625,30401.132812
4,348626,3810.147461


In [185]:
# In GBM you can get some negative predictions:
print sample_submission[sample_submission['y'] < 0]

         Num          y
399   349692 -49.195480
625   349918 -82.665092
851   350364 -49.195480
1077  350590 -49.195480
1078  350591 -51.080593
1523  351036 -40.327545
1524  351037 -51.080593


In [186]:
sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)

In [187]:
sample_submission.to_csv("baseline_submission.tsv", sep=',', index=False)

In [ ]:
import numpy as np
range(100, 501, 40)